In [1]:
import pandas as pd
from collections import defaultdict

In [2]:
list_data_file_path = "resources/List Price.csv"
sale_data_file_path = "resources/Sale Price.csv"

In [3]:
raw_list_df = pd.read_csv(list_data_file_path)
raw_sale_df = pd.read_csv(sale_data_file_path)

In [4]:
raw_list_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2017-11-04,2017-11-11,2017-11-18,2017-11-25,2017-12-02,...,2021-04-24,2021-05-01,2021-05-08,2021-05-15,2021-05-22,2021-05-29,2021-06-05,2021-06-12,2021-06-19,2021-06-26
0,102001,0,United States,Country,NaN,275448.0,275448.0,275448.0,274975.0,274825.0,...,368921.0,371946.0,374750.0,377225.0,378934.0,381434.0,382934.0,384209.0,385000.0,385000.0
1,394913,1,"New York, NY",Msa,NY,526850.0,527331.0,527584.0,526065.0,525815.0,...,672375.0,672500.0,672250.0,671000.0,669250.0,666500.0,661750.0,656750.0,652250.0,649750.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,812252.0,813533.0,813533.0,815583.0,816608.0,...,1012000.0,1005750.0,999500.0,999750.0,999750.0,999500.0,999475.0,999225.0,999225.0,998350.0
3,394463,3,"Chicago, IL",Msa,IL,319556.0,317780.0,315751.0,311972.0,308649.0,...,349975.0,350999.0,352349.0,354599.0,357074.0,358575.0,359475.0,358575.0,357350.0,356100.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,347537.0,346298.0,345034.0,343958.0,343704.0,...,377784.0,380495.0,383493.0,386245.0,387745.0,389745.0,391748.0,392495.0,392498.0,391747.0


In [5]:
raw_sale_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,2008-02-23,2008-03-01,2008-03-08,2008-03-15,2008-03-22,...,2021-03-20,2021-03-27,2021-04-03,2021-04-10,2021-04-17,2021-04-24,2021-05-01,2021-05-08,2021-05-15,2021-05-22
0,102001,0,United States,Country,NaN,191138.0,192225.0,192225.0,192475.0,194750.0,...,299500.0,301625.0,306484.0,309234.0,312359.0,314859.0,315000.0,317500.0,319363.0,320862.0
1,394913,1,"New York, NY",Msa,NY,NaN,NaN,NaN,NaN,NaN,...,476225.0,476250.0,478750.0,483738.0,486238.0,491238.0,493738.0,500000.0,507500.0,510000.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,516750.0,515500.0,510688.0,509688.0,502188.0,...,776875.0,778125.0,779375.0,783063.0,791688.0,801750.0,803125.0,810688.0,811188.0,816125.0
3,394463,3,"Chicago, IL",Msa,IL,247988.0,252488.0,251613.0,251000.0,249750.0,...,263963.0,267213.0,271713.0,275088.0,275375.0,277442.0,280442.0,285004.0,290004.0,294188.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,143466.0,144591.0,146463.0,149213.0,150713.0,...,317938.0,319750.0,319821.0,324321.0,329571.0,330821.0,332375.0,337431.0,340431.0,345431.0


In [6]:
def condense_date_columns(df):
    # Grab non-date columns from dataframe
    non_date_columns = [x for x in df.columns if x[0].isalpha()]
    
    # Create data and non-date dataframes based on original df
    non_date_df = df[non_date_columns]
    date_df = df.drop(columns=non_date_columns)
    
    # Initialize final data dictionary
    data = defaultdict(list)
    
    # Loop through dates
    for col_date in date_df.columns:
        
        # Loop through values for each date
        for i, x in enumerate(list(date_df[col_date])):
            
            # Loop through non-date values
            for col_non_date in non_date_df.columns:
                
                # Append non-date value to non-date column
                data[str(col_non_date)].append(non_date_df[col_non_date][i])
            
            # Append date and price values
            data["date"].append(str(col_date))
            data["price"].append(x)
            
    return pd.DataFrame(data)

In [7]:
final_list_df = condense_date_columns(raw_list_df)
final_sale_df = condense_date_columns(raw_sale_df)

In [8]:
print(final_list_df.shape)
final_list_df.head()

(18527, 7)


,RegionID,SizeRank,RegionName,RegionType,StateName,date,price
0,102001,0,United States,Country,NaN,2017-11-04,275448.0
1,394913,1,"New York, NY",Msa,NY,2017-11-04,526850.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,2017-11-04,812252.0
3,394463,3,"Chicago, IL",Msa,IL,2017-11-04,319556.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,2017-11-04,347537.0


In [9]:
print(final_sale_df.shape)
final_sale_df.head()

(67124, 7)


,RegionID,SizeRank,RegionName,RegionType,StateName,date,price
0,102001,0,United States,Country,NaN,2008-02-23,191138.0
1,394913,1,"New York, NY",Msa,NY,2008-02-23,NaN
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,2008-02-23,516750.0
3,394463,3,"Chicago, IL",Msa,IL,2008-02-23,247988.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,2008-02-23,143466.0


In [10]:
final_list_df.to_csv("resources/final_list_price_data.csv", index=False)
final_sale_df.to_csv("resources/final_sale_price_data.csv", index=False)